# Introduction

This notebook shows how to replace the `VolatileMemoryStore` memory storage used in a [previous notebook](./06-memory-and-embeddings.ipynb) with a `WeaviateMemoryStore`.

`WeaviateMemoryStore` is an example of a persistent (i.e. long-term) memory store backed by the Weaviate vector database.

# About Weaviate

[Weaviate](https://weaviate.io/) is an open-source vector database designed to scale seamlessly into billions of data objects. This implementation supports hybrid search out-of-the-box (meaning it will perform better for keyword searches).

You can run Weaviate in 5 ways:

- **SaaS** – with [Weaviate Cloud Services (WCS)](https://weaviate.io/pricing).

  WCS is a fully managed service that takes care of hosting, scaling, and updating your Weaviate instance. You can try it out for free with a sandbox that lasts for 14 days.

  To set up a SaaS Weaviate instance with WCS:

  1.  Navigate to [Weaviate Cloud Console](https://console.weaviate.cloud/).
  2.  Register or sign in to your WCS account.
  3.  Create a new cluster with the following settings:
      - `Subscription Tier` – Free sandbox for a free trial, or contact [hello@weaviate.io](mailto:hello@weaviate.io) for other options.
      - `Cluster name` – a unique name for your cluster. The name will become part of the URL used to access this instance.
      - `Enable Authentication?` – Enabled by default. This will generate a static API key that you can use to authenticate.
  4.  Wait for a few minutes until your cluster is ready. You will see a green tick ✔️ when it's done. Copy your cluster URL.

- **Hybrid SaaS**

  > If you need to keep your data on-premise for security or compliance reasons, Weaviate also offers a Hybrid SaaS option: Weaviate runs within your cloud instances, but the cluster is managed remotely by Weaviate. This gives you the benefits of a managed service without sending data to an external party.

  The Weaviate Hybrid SaaS is a custom solution. If you are interested in this option, please reach out to [hello@weaviate.io](mailto:hello@weaviate.io).

- **Self-hosted** – with a Docker container

  To set up a Weaviate instance with Docker:

  1. [Install Docker](https://docs.docker.com/engine/install/) on your local machine if it is not already installed.
  2. [Install the Docker Compose Plugin](https://docs.docker.com/compose/install/)
  3. Download a `docker-compose.yml` file with this `curl` command:

     ```
     curl -o docker-compose.yml "https://configuration.weaviate.io/v2/docker-compose/docker-compose.yml?modules=standalone&runtime=docker-compose&weaviate_version=v1.19.6"
     ```

     Alternatively, you can use Weaviate's docker compose [configuration tool](https://weaviate.io/developers/weaviate/installation/docker-compose) to generate your own `docker-compose.yml` file.

  4. Run `docker compose up -d` to spin up a Weaviate instance.

     > To shut it down, run `docker compose down`.

- **Self-hosted** – with a Kubernetes cluster

  To configure a self-hosted instance with Kubernetes, follow Weaviate's [documentation](https://weaviate.io/developers/weaviate/installation/kubernetes).|

- **Embedded** - start a weaviate instance right from your application code using the client library
   
  This code snippet shows how to instantiate an embedded weaviate instance and upload a document:

  ```python
  import weaviate
  from weaviate.embedded import EmbeddedOptions

  client = weaviate.Client(
    embedded_options=EmbeddedOptions()
  )

  data_obj = {
    "name": "Chardonnay",
    "description": "Goes with fish"
  }

  client.data_object.create(data_obj, "Wine")
  ```
  
  Refer to the [documentation](https://weaviate.io/developers/weaviate/installation/embedded) for more details about this deployment method.

# Setup

In [1]:
!pip install semantic-kernel==0.3.0.dev0
!pip install weaviate-client
!pip install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.3/94.3 kB 869.5 kB/s eta 0:00:001m797.8 kB/s eta 0:00:01
  Attempting uninstall: semantic-kernel
    Found existing installation: semantic-kernel 0.2.9.dev0
    Uninstalling semantic-kernel-0.2.9.dev0:
      Successfully uninstalled semantic-kernel-0.2.9.dev0


## OS-specific notes:
* if you run into SSL errors when connecting to OpenAI on macOS, see this issue for a [potential solution](https://github.com/microsoft/semantic-kernel/issues/627#issuecomment-1580912248)
* on Windows, you may need to run Docker Desktop as administrator

In [2]:
from typing import Tuple

import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    OpenAITextCompletion,
    OpenAITextEmbedding,
)

import os

First, we instantiate the Weaviate memory store. Uncomment ONE of the options below, depending on how you want to use Weaviate:
* from a Docker instance
* from WCS
* directly from the client (embedded Weaviate), which works on Linux only at the  moment

In [3]:
from semantic_kernel.connectors.memory import weaviate_memory_store
from dotenv import load_dotenv

load_dotenv(override=True)

# Using Docker
config = weaviate_memory_store.WeaviateConfig(url="http://localhost:8080")

# Using WCS. Make sure the environment variables `WEAVIATE_URL` and `WEAVIATE_API_KEY` were set in the `.env` file.
#config = weaviate_memory_store.WeaviateConfig(
#    url=os.getenv("WEAVIATE_URL"),
#    api_key=os.getenv("WEAVIATE_API_KEY")
#)

# Using Embedded Weaviate
#config = weaviate_memory_store.WeaviateConfig(use_embed=True)

store = weaviate_memory_store.WeaviateMemoryStore(config=config)
store.client.schema.delete_all()

Then, we register the memory store to the kernel:

In [4]:
kernel = sk.Kernel()

api_key, org_id = sk.openai_settings_from_dot_env()
kernel.add_text_completion_service(
    "dv", OpenAITextCompletion("text-davinci-003", api_key, org_id)
)
kernel.add_text_embedding_generation_service(
    "ada", OpenAITextEmbedding("text-embedding-ada-002", api_key, org_id)
)

kernel.register_memory_store(memory_store=store)
kernel.import_skill(sk.core_skills.TextMemorySkill())

{'recall': <semantic_kernel.orchestration.sk_function.SKFunction at 0x103c21250>,
 'save': <semantic_kernel.orchestration.sk_function.SKFunction at 0x105926710>}

# Manually adding memories


Let's create some initial memories "About Me". We can add memories to our weaviate memory store by using `save_information_async`

In [5]:
COLLECTION = "AboutMe"

In [6]:
async def populate_memory(kernel: sk.Kernel) -> None:
    # Add some documents to the semantic memory
    await kernel.memory.save_information_async(
        COLLECTION, id="info1", text="My name is Andrea"
    )
    await kernel.memory.save_information_async(
        COLLECTION, id="info2", text="I currently work as a tour guide"
    )
    await kernel.memory.save_information_async(
        COLLECTION, id="info3", text="I've been living in Seattle since 2005"
    )
    await kernel.memory.save_information_async(
        COLLECTION, id="info4", text="I visited France and Italy five times since 2015"
    )
    await kernel.memory.save_information_async(
        COLLECTION, id="info5", text="My family is from New York"
    )

Searching is done through `search_async`:

In [7]:
async def search_memory_examples(kernel: sk.Kernel) -> None:
    questions = [
        "what's my name",
        "where do I live?",
        "where's my family from?",
        "where have I traveled?",
        "what do I do for work",
    ]

    for question in questions:
        print(f"Question: {question}")
        result = await kernel.memory.search_async(COLLECTION, question)
        print(f"Answer: {result[0].text}\n")

Let's see the results of the functions:

In [8]:
print("Populating memory...")
await populate_memory(kernel)

print("Asking questions... (manually)")
await search_memory_examples(kernel)

Populating memory...
Asking questions... (manually)
Question: what's my name
Answer: My name is Andrea

Question: where do I live?
Answer: I've been living in Seattle since 2005

Question: where's my family from?
Answer: My family is from New York

Question: where have I traveled?
Answer: I visited France and Italy five times since 2015

Question: what do I do for work
Answer: I currently work as a tour guide



Here's how to use the weaviate memory store in a chat application:

In [9]:
async def setup_chat_with_memory(
    kernel: sk.Kernel,
) -> Tuple[sk.SKFunctionBase, sk.SKContext]:
    sk_prompt = """
    ChatBot can have a conversation with you about any topic.
    It can give explicit instructions or say 'I don't know' if
    it does not have an answer.

    Information about me, from previous conversations:
    - {{$fact1}} {{recall $fact1}}
    - {{$fact2}} {{recall $fact2}}
    - {{$fact3}} {{recall $fact3}}
    - {{$fact4}} {{recall $fact4}}
    - {{$fact5}} {{recall $fact5}}

    Chat:
    {{$chat_history}}
    User: {{$user_input}}
    ChatBot: """.strip()

    chat_func = kernel.create_semantic_function(
        sk_prompt, max_tokens=200, temperature=0.8
    )

    context = kernel.create_new_context()
    context["fact1"] = "what is my name?"
    context["fact2"] = "where do I live?"
    context["fact3"] = "where's my family from?"
    context["fact4"] = "where have I traveled?"
    context["fact5"] = "what do I do for work?"

    context[sk.core_skills.TextMemorySkill.COLLECTION_PARAM] = COLLECTION
    context[sk.core_skills.TextMemorySkill.RELEVANCE_PARAM] = 0.8

    context["chat_history"] = ""

    return chat_func, context

In [10]:
async def chat(
    kernel: sk.Kernel, chat_func: sk.SKFunctionBase, context: sk.SKContext
) -> bool:
    try:
        user_input = input("User:> ")
        context["user_input"] = user_input
        print(f"User:> {user_input}")
    except KeyboardInterrupt:
        print("\n\nExiting chat...")
        return False
    except EOFError:
        print("\n\nExiting chat...")
        return False

    if user_input == "exit":
        print("\n\nExiting chat...")
        return False

    answer = await kernel.run_async(chat_func, input_vars=context.variables)
    context["chat_history"] += f"\nUser:> {user_input}\nChatBot:> {answer}\n"

    print(f"ChatBot:> {answer}")
    return True

In [11]:
print("Setting up a chat (with memory!)")
chat_func, context = await setup_chat_with_memory(kernel)

print("Begin chatting (type 'exit' to exit):\n")
chatting = True
while chatting:
    chatting = await chat(kernel, chat_func, context)

Setting up a chat (with memory!)
Begin chatting (type 'exit' to exit):

User:> hi
ChatBot:>  Hi there! How can I help you?
User:> where is my family from?
ChatBot:>  Your family is from New York.
User:> tell me a joke
ChatBot:>  Why did the chicken cross the playground?
    To get to the other slide!
User:> bye!
ChatBot:> Error: (<ErrorCodes.ServiceError: 6>, 'OpenAI service failed to complete the prompt', RateLimitError(message='The server had an error while processing your request. Sorry about that!', http_status=429, request_id=None))
User:> exit


Exiting chat...


# Adding documents to your memory

Create a dictionary to hold some files. The key is the hyperlink to the file and the value is the file's content:

In [12]:
github_files = {}
github_files[
    "https://github.com/microsoft/semantic-kernel/blob/main/README.md"
] = "README: Installation, getting started, and how to contribute"
github_files[
    "https://github.com/microsoft/semantic-kernel/blob/main/dotnet/notebooks/02-running-prompts-from-file.ipynb"
] = "Jupyter notebook describing how to pass prompts from a file to a semantic skill or function"
github_files[
    "https://github.com/microsoft/semantic-kernel/blob/main/dotnet/notebooks/00-getting-started.ipynb"
] = "Jupyter notebook describing how to get started with the Semantic Kernel"
github_files[
    "https://github.com/microsoft/semantic-kernel/tree/main/prompts/samples/ChatPlugin/ChatGPT"
] = "Sample demonstrating how to create a chat skill interfacing with ChatGPT"
github_files[
    "https://github.com/microsoft/semantic-kernel/blob/main/dotnet/src/SemanticKernel/Memory/Volatile/VolatileMemoryStore.cs"
] = "C# class that defines a volatile embedding store"
github_files[
    "https://github.com/microsoft/semantic-kernel/tree/main/samples/dotnet/KernelHttpServer/README.md"
] = "README: How to set up a Semantic Kernel Service API using Azure Function Runtime v4"
github_files[
    "https://github.com/microsoft/semantic-kernel/tree/main/samples/apps/chat-summary-webapp-react/README.md"
] = "README: README associated with a sample starter react-based chat summary webapp"

Use `save_reference_async` to save the file:

In [13]:
COLLECTION = "SKGitHub"

print(
    "Adding some GitHub file URLs and their descriptions to a volatile Semantic Memory."
)
i = 0
for entry, value in github_files.items():
    await kernel.memory.save_reference_async(
        collection=COLLECTION,
        description=value,
        text=value,
        external_id=entry,
        external_source_name="GitHub",
    )
    i += 1
    print("  URL {} saved".format(i))

Adding some GitHub file URLs and their descriptions to a volatile Semantic Memory.
  URL 1 saved
  URL 2 saved
  URL 3 saved
  URL 4 saved
  URL 5 saved
  URL 6 saved
  URL 7 saved


Use `search_async` to ask a question:

In [14]:
ask = "I love Jupyter notebooks, how should I get started?"
print("===========================\n" + "Query: " + ask + "\n")

memories = await kernel.memory.search_async(
    COLLECTION, ask, limit=5, min_relevance_score=0.77
)

i = 0
for memory in memories:
    i += 1
    print(f"Result {i}:")
    print("  URL:     : " + memory.id)
    print("  Title    : " + memory.description)
    print("  Relevance: " + str(memory.relevance))
    print()

Query: I love Jupyter notebooks, how should I get started?

Result 1:
  URL:     : https://github.com/microsoft/semantic-kernel/blob/main/samples/notebooks/dotnet/00-getting-started.ipynb
  Title    : Jupyter notebook describing how to get started with the Semantic Kernel
  Relevance: 0.9338777661323547

Result 2:
  URL:     : https://github.com/microsoft/semantic-kernel/blob/main/samples/notebooks/dotnet/02-running-prompts-from-file.ipynb
  Title    : Jupyter notebook describing how to pass prompts from a file to a semantic skill or function
  Relevance: 0.9082032442092896

Result 3:
  URL:     : https://github.com/microsoft/semantic-kernel/blob/main/README.md
  Title    : README: Installation, getting started, and how to contribute
  Relevance: 0.9042205810546875

Result 4:
  URL:     : https://github.com/microsoft/semantic-kernel/tree/main/samples/apps/chat-summary-webapp-react/README.md
  Title    : README: README associated with a sample starter react-based chat summary webapp
  R